### Load the necessary libraries and modules


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import (
    SelectKBest, f_classif, chi2, mutual_info_classif,
    RFE, RFECV, SelectFromModel, VarianceThreshold
)
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OrdinalEncoder
from sklearn.metrics import classification_report, accuracy_score
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr
import warnings
warnings.filterwarnings('ignore')
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import shap

# Load the dataset

- The data has 100 rows and 154 columns with 151 numerical columns, 1 categorical column(RATING_TYPE), 1 unstructured data column(string_values) and 1 categorical with order column(Rating)

In [ ]:
data_url = "https://github.com/Banking-Analytics-Lab/MultimodalFusionRatings/blob/main/Data/Artificial_Data.xlsx?raw=true"

# Load the dataset

df_1 = pd.read_excel(data_url)

print("dataset is loaded successfully.\n")

# checking first five rows of the data

print(df_1.head())

In [ ]:
# Inspecting the data

# Extract the columns and data types
- 151 numerical columns belongs to 'float64' dtype while the remaining categorical and text colums are objects.

In [ ]:

col_data_types = df_1.dtypes
print(f"The data types of the columns are:\n\n{col_data_types}\n")

# Basic descriptive statistics

In [ ]:
print(f"Descriptive Stats for the dataframe:\n\n{df_1.describe()}\n")

# Observations based on the data description
- Based on the descriptive stats, it seems the data is more or less pre-processed and doesn't have any significant outliers. The standard deviation for majority of the columns are with in +1SD. Min and Max values are b/w -3 to 3. So, the scaling of the variables is already done in the data.

In [ ]:
# We will check for outliers in the columns to understand the data quality.
for col in df_1.select_dtypes(include='float64').columns:
    Q1 = df_1[col].quantile(0.25)
    Q3 = df_1[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers
    outliers = df_1[(df_1[col] < lower_bound) | (df_1[col] > upper_bound)]

    print(f"\nOutliers in column '{col}':")
    print(outliers[[col]])

# Understanding the proprotion of the outliers in each column


In [ ]:
outlier_perc = {}

for col in df_1.select_dtypes(include='float64').columns:

    Q1 = df_1[col].quantile(0.25)
    Q3 = df_1[col].quantile(0.75)

    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    l_outlier = len(df_1.loc[(df_1[col] < lower_bound)  ,col])
    u_outlier = len(df_1.loc[(df_1[col] > upper_bound)  ,col])

    outlier_count = l_outlier + u_outlier

    percentage_outlier = outlier_count * 100/len(df_1)


    outlier_perc[col] = percentage_outlier

    print(f"{col}:{(percentage_outlier)}")

# Distribution of outlier percentage across the data
- Except for one variable which have 5% and 7 variables which have 3% outliers,
the remianing variables are well with in the tolerance limit. So, outliers are not creating any major issues in the data.

In [ ]:

outlier_series = pd.Series(outlier_perc).sort_values(ascending=False)

# Plot
plt.figure(figsize=(10, 6))
outlier_series.plot(kind='bar', color='salmon', edgecolor='black')
plt.ylabel("Outlier Percentage")
plt.title("Outlier Percentage by Column")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Check for missing values

- There are no missing values in the data. The data seems ideal which is not the case usually in real world.

In [ ]:

print(f"\nMissing Values:")
missing_data = df_1.isnull().sum()
missing_perc = (missing_data / len(df_1)) * 100
missing_df = pd.DataFrame({
    'missing_count': missing_data,
    'missing_percentage': missing_perc
})
missing_df = missing_df[missing_df['missing_count'] > 0].sort_values('missing_count', ascending=False)
print(missing_df)

# Check Skewness in the data
- As per the stats of the data skewness, the data is fairly symmetric and therefore requires no intervention.

In [ ]:
skew_data = df_1.select_dtypes(include='float64').skew().sort_values(ascending=False)

print(f"The skew data is:\n{skew_data.describe()}")

# correlation matrix
- No pairs are highly correlated as analysed by setting up a threshold of 0.9. Therefore, there is no strong relationship among the features.
- There are 0 high correlation pairs as analysed below. Changing the threshold to 0.4 results in 2 pairs.

In [ ]:
corr_df = df_1.select_dtypes(include='float64').corr()

# create correlation matrix as a dataframe
corr_matrix = pd.DataFrame(corr_df)

#print(corr_matrix.head())

# Set a threshold
threshold = 0.9

# Find correlated pairs above the threshold
high_corr = (corr_df.abs() > threshold) & (corr_df.abs() < 1.0)

# Extract the column pairs
high_corr_pairs = []

for col in high_corr.columns:
    corr_features = high_corr.index[high_corr[col]].tolist()

    for row in corr_features:
        if (row, col) not in high_corr_pairs and (col, row) not in high_corr_pairs:
            high_corr_pairs.append((row, col))


# Print the pairs and their correlation values
for row, col in high_corr_pairs:
    print(f"{row} ↔ {col} : correlation = {corr_df.loc[row, col]:.2f}")


# visualize correlation features

In [ ]:
# Step 3: Visualize it as a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_df, cmap='coolwarm', center=0, annot=False, fmt=".2f")
plt.title("Correlation Matrix of Numeric Columns")
plt.tight_layout()
plt.show()

# Target Variable Selection
- Based on the analysis of the data, the target variable will be 'Rating'. The 'RATING_TYPE' variable which represents rating agencies can be used if the rating agencies will have different 'Rating' standards. But, in this data, the ratings of each rating agencies are of the same type. If they would have been different where Moody's have different rating scale as compared to S&P's and Fitch, then we would have chosen a new variable- 'Risk_Scale' as the target variable to eliminate the subjectivity of the rating agencies using a matrix mapping of the different rating agencies.
But since the data consists of same rating standard for all the 3 rating agenceis, it doesn't need a new target variable. We will convert the 'Rating' to numeric rating after mapping the original rating to a risk scale.

# Feature Selection
- Based on the target variable, correlation and the domain knowledg, the feature variables will be the financial ratios and unstructured text as present in the data.
The rest of the ratios are not important considering that the company credit rating is dependent on the financial information. The stock index, bond index returns are important when the target variable would have been 'Future_Performance'.

In [ ]:
# Taking only financial rations as structured variables as the other like index returns/bond market returns are not
#important
structured_cols = ['bm','evm','pe_op_basic','pe_op_dil','pe_exi','pe_inc','ps','pcf','npm','opmbd','opmad','gpm','ptpm','cfm','roa','roe','roce','aftret_eq','aftret_invcapx','aftret_equity','GProf','equity_invcap','debt_invcap','totdebt_invcap','capital_ratio','cash_lt','debt_at','debt_ebitda','lt_debt','cash_debt','lt_ppent','dltt_be','debt_assets','debt_capital','de_ratio','at_turn','rect_turn','pay_turn','sale_invcap','sale_equity','rd_sale','adv_sale','staff_sale','accrual'
]

In [ ]:
# New dataframe with Target Variable as Rating, financial ratios as structured ratios and
#string values as unstructured(text) variable

df = df_1[['Rating'] + structured_cols + ['string_values']]

# Encode the 'Rating' Variable to numerical rating for modelling
- Since the rating has inherent order and in business, it is interpreted as a symbol of risk measurement, it is essential to map them to risk and then to a numeric value.
- Since we only have ratings from AAA to BB, we are mapping them from Lowest Risk to Highest Risk. The Highest Risk should not be taken in literal sense as the highest risk usaully occurs for C and below Rating.
- For eg: AAA is the highest rating and lowest risk. The jump b/w AAA and the second best rating AA+ is different than the jump b/w AA+ and AA.

In [ ]:
rating_to_risk = {
                'AAA': 'Lowest Risk',
                'AA+': 'Low Risk',
                'AA': 'Low Risk',
                'A+': 'Medium Risk',
                'A': 'Medium Risk',
                'BBB+': 'High Risk',
                'BBB': 'Highest Risk',
                'BB': 'Highest Risk'
            }

# Risk category to numerical mapping (ordered by risk level)
risk_to_numeric = {
    'Lowest Risk': 0,
    'Low Risk': 1,
    'Medium Risk': 2,
    'High Risk': 3,
    'Highest Risk': 4,
    'In Default': 5
}

# for rating column, map it to risk category
if 'Rating' in df.columns:
    print("Converting credit ratings to risk categories...")

    # Convert credit rating to risk category
    risk_cat = df['Rating'].map(rating_to_risk)

    # Now convert risk category to numeric
    rating_numeric = risk_cat.map(risk_to_numeric)

   # Credit Rating -> Risk Category -> Numerical Mapping
    for rating, risk in rating_to_risk.items():
        numeric = risk_to_numeric[risk]
        print(f"  {rating:>3} -> {risk:<12} -> {numeric}")

   # Target distribution after encoding
    risk_counts = risk_cat.value_counts()
    numeric_counts = pd.Series(rating_numeric).value_counts().sort_index()

    for risk_cat, count in risk_counts.items():
        numeric_val = risk_to_numeric[risk_cat]
        print(f"  {risk_cat:<12} ({numeric_val}): {count:>4} samples")


# Distribution of Target Variable
- The distribution reflects that the samples are not severly imbalanced. Numeric Rating with 0 & 3
have 10 samples each and Numeric Rating with 1, 2 & 4 have 25,26 and 29 samples respectively.

In [ ]:
# Plot distribution of risk categories
plt.figure(figsize=(8, 4))
sns.barplot(x=risk_counts.index, y=risk_counts.values, palette="Blues_d")
plt.title("Distribution of Risk Categories")
plt.ylabel("Number of Samples")
plt.xlabel("Risk Category")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plot distribution of encoded numeric risk
plt.figure(figsize=(6, 4))
sns.barplot(x=numeric_counts.index.astype(str), y=numeric_counts.values, palette="Oranges_d")
plt.title("Distribution of Encoded Risk (Numeric)")
plt.ylabel("Number of Samples")
plt.xlabel("Encoded Risk Level")
plt.tight_layout()
plt.show()

In [ ]:
# Replace the original 'Rating' column with 'rating_numeric'
df['rating_numeric'] = rating_numeric

# Drop the original 'Rating' column
df = df.drop(columns=['Rating'])


# Correlation of the new input features with the new target variable

In [ ]:

# Randomly select 4 numeric columns (excluding the target)
feature_cols = df.select_dtypes(include='float64').columns
sampled_cols = random.sample(list(feature_cols), 4)

# Set up 2x2 subplot grid
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('Box plot of cols with target variable', fontsize=14)

# Plot each feature in a subplot
for ax, col in zip(axes.flatten(), sampled_cols):
    sns.boxplot(x='rating_numeric', y=col, data=df, ax=ax)
    ax.set_title(f'{col}')
    ax.set_xlabel('Risk Rating')
    ax.set_ylabel(col)

plt.tight_layout(rect=[0, 0, 1, 0.95])  # Leave space for the suptitle
plt.show()

# NLP for converting the unstructured text column with embeddings
- A quick check on the stats of the column 'string_values' suggests that the texts are repeating and there are only 5 unique text statments. This may be as the data is artifical and does not cover the real-world data.
- Sentiment analysis on such text may not yield good results and therefore, we will use embeddings to capture the semantics to make better use of the information.
- There's not enough data to do topic modelling and hence it is not useful here.

In [ ]:
# Basic descriptive stats
df['string_values'].describe()

In [ ]:
# Distribution of the text data

# Count of the unique values of the text data
- Since most of the text data is neutral to positive, there is non- significant variation in the sentiment of the data and therefore sentiment analysis will not suit this data


In [ ]:
df['string_values'].value_counts().head(10)

# Sentiment Analysis
- By using https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english model, I have tried calculating the sentiment label and score, it doesn't have significant variation.
    and therefore, sentiment analysis will not be useful in prediction.

In [ ]:
from transformers import pipeline

# Load pretrained sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

# Example text column
texts = df['string_values'].tolist()

# Get sentiment predictions
sentiments = sentiment_pipeline(texts)

# Extract scores into DataFrame
df['sentiment_label'] = [s['label'] for s in sentiments]
df['sentiment_score'] = [s['score'] for s in sentiments]

# Transformer based embeddings
- We will load the small model 'all-MiniLM-L6-v2' to calculated the embeddings of the text.
But since the dimension of the vector is 384, we will reduce them to 10 dimensions using PCA and scale the values
so that they match the scale of the other features.


In [ ]:

# 1. Load model and encode
model = SentenceTransformer('all-MiniLM-L6-v2')
texts = df['string_values'].astype(str).tolist()
embeddings = model.encode(texts, show_progress_bar=True)

# 2. Reduce dimensionality with PCA
n_components = 10
pca = PCA(n_components=n_components)
embeddings_pca = pca.fit_transform(embeddings)

# 3. Scale PCA output to range [-3, 3]
scaler = MinMaxScaler(feature_range=(-3, 3))
embeddings_scaled = scaler.fit_transform(embeddings_pca)

# 4. Convert to DataFrame and merge back to df
embedding_df = pd.DataFrame(embeddings_scaled, columns=[f'emb_{i+1}' for i in range(n_components)])
df = pd.concat([df.reset_index(drop=True), embedding_df], axis=1)


# Predictive Modelling
- Since we are dealing with credit ratings, we will use classification models to predict the ratings.
- we have made numeric classes of ratings after mapping the original ratings to risk categories.
- We will do GBM model training for this exercise.

In [ ]:
# starting with structured features only


X = df[structured_cols]
y = df['rating_numeric']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


In [ ]:
# define models and hyperparameter grid

models = {
    'GBM': {
        'model': GradientBoostingClassifier(random_state=42),
        'params': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7]
        }
    },
    'LightGBM': {
        'model': LGBMClassifier(random_state=42),
        'params': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.05, 0.1],
            'num_leaves': [15, 31, 63],
            'max_depth': [-1, 5, 10]
        }
    }

}


In [ ]:
# loop through models and tune hyperparameters

structured_results = []

for name, mp in models.items():
    print(f"Running model: {name}")

    search = RandomizedSearchCV(
        mp['model'],
        mp['params'],
        n_iter=10,
        cv=5,
        scoring='accuracy',
        verbose=1,
        n_jobs=-1,
        random_state=42
    )

    search.fit(X_train, y_train)

    print(f"Best parameters for {name}: {search.best_params_}")
    best_model = search.best_estimator_

    # Evaluate
    y_pred = best_model.predict(X_test)
    print(f"Accuracy on test set for {name}: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred))

    structured_results.append({
        'Model': name,
        'Accuracy': acc
    })


In [ ]:
structured_results_df = pd.DataFrame(structured_results).sort_values(by='Accuracy', ascending=False)
print("\nStructured Feature Model Comparison:")
print(structured_results_df)

In [ ]:
# Now training using text-only embedding features

In [ ]:
# text embeddings features dataframe
X_text = df[[f"emb_{i}" for i in range(1, 11)]]
y = df['rating_numeric']







In [ ]:
# 2. Train-test split
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(
    X_text, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
X_text

In [ ]:
# 3. Define models and hyperparameters
models = {
    'GBM_text': {
        'model': GradientBoostingClassifier(random_state=42),
        'params': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7]
        }
    },
    'LightGBM_text': {
        'model': LGBMClassifier(random_state=42),
        'params': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.05, 0.1],
            'num_leaves': [15, 31, 63],
            'max_depth': [-1, 5, 10]
        }
    }
}

In [ ]:
# 4. Train, tune and evaluate
text_results = []

for name, mp in models.items():
    print(f"\nTraining {name} on text features...")

    search = RandomizedSearchCV(
        estimator=mp['model'],
        param_distributions=mp['params'],
        n_iter=10,
        cv=5,
        scoring='accuracy',
        verbose=1,
        n_jobs=-1,
        random_state=42
    )

    search.fit(X_train_text, y_train_text)
    best_model = search.best_estimator_

    print(f"Best params for {name}: {search.best_params_}")

    y_pred_text = best_model.predict(X_test_text)

    accuracy_text = accuracy_score(y_test_text, y_pred_text)

    print(f"Accuracy for {name}: {accuracy_text:.4f}")
    print(classification_report(y_test_text, y_pred_text))

    text_results.append({
        'Model': name,
        'Accuracy': accuracy_text
    })



In [ ]:
# 5. Summary
text_results_df = pd.DataFrame(text_results).sort_values(by='Accuracy', ascending=False)
print("\nText Feature Model Comparison:")
print(text_results_df)

In [ ]:
# now train model with structured and text columns

In [ ]:
# Step 1: Combine structured + text features
X_combined = pd.concat([X, X_text], axis=1)

# Step 2: Train-test split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(
    X_combined, y, test_size=0.2, stratify=y, random_state=42
)

# Step 3: Define models and hyperparameter grids
models = {
    'GBM_combined': {
        'model': GradientBoostingClassifier(random_state=42),
        'params': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7]
        }
    },
    'LightGBM_combined': {
        'model': LGBMClassifier(random_state=42),
        'params': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.05, 0.1],
            'num_leaves': [15, 31, 63],
            'max_depth': [-1, 5, 10]
        }
    }
}

# Step 4: Train and evaluate each model
combined_results = []

for name, mp in models.items():
    print(f"\nTraining {name} on combined features...")

    search = RandomizedSearchCV(
        estimator=mp['model'],
        param_distributions=mp['params'],
        n_iter=10,
        cv=5,
        scoring='accuracy',
        verbose=1,
        n_jobs=-1,
        random_state=42
    )

    search.fit(X_train_comb, y_train_comb)
    best_model = search.best_estimator_

    print(f"Best parameters for {name}: {search.best_params_}")

    y_pred = best_model.predict(X_test_comb)

    acc = accuracy_score(y_test_comb, y_pred)
    print(f"Accuracy on test set: {acc:.4f}")
    print(classification_report(y_test_comb, y_pred))

    combined_results.append({
        'Model': name,
        'Accuracy': acc
    })

# Step 5: Summary table
combined_results_df = pd.DataFrame(combined_results).sort_values(by='Accuracy', ascending=False)
print("\Combined Feature Model Comparison:")
print(combined_results_df)


# create confusion matrix based on the combined data
- since the overall data size is jsut 100 rows, the training data is not enough to train the model. As a result, we can see the model is not able to predict the right class for different class ratings.

In [ ]:
# Generate confusion matrix
cm = confusion_matrix(y_test_comb, y_pred)

# Display it
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

# Comparison Report of the performance of models across structured only, text only and combined features


In [ ]:
structured_results_df['DataType'] = 'Structured'
text_results_df['DataType'] = 'Text'
combined_results_df['DataType'] = 'Combined'

performance_df = pd.concat([
    structured_results_df,
    text_results_df,
    combined_results_df
], ignore_index=True)


# Visualize the performance
- We can see the performance of the GBM model has increased after considering the unstructured text into the model.

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=performance_df, x='Model', y='Accuracy', hue='DataType')
plt.title('Model Performance Comparison')
plt.xticks(rotation=45)
plt.ylim(0, 1)
plt.tight_layout()
plt.show()


#SHAP analysis
- side by side view of which features impact different rating classes

In [ ]:
# feature importance using SHAP

# Using GBM for the combined data
explainer = shap.TreeExplainer(best_model)

shap_values = explainer.shap_values(X_combined)


# Plot summary
shap.summary_plot(shap_values, X_combined, plot_type="bar")


In [ ]:
# comment

# Feature Importance by each class type of the rating
- The rating of low risk class doesn't have much impact from the leverage ratios while the high risk classes does have more impact from the leverage ratios. This confirms the trend that is usaully scene in the real world.

In [ ]:
# Split shap values by class
shap_values_by_class = [shap_values[:, :, i] for i in range(shap_values.shape[2])]

# Plot for class 0
shap.summary_plot(shap_values_by_class[0], X_combined, plot_type='bar')